# Dataset 2 - Clasificación multiclase

###En el dataset "MNIST in CSV para predecir números de 0 al 9" se aplicacrá la regresión logística onevsall. Se comenzará haciendo el preprocesamiento del dataset con Pandas.

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. Librerías

In [79]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

#Libreria pandas y scikitlearn para el preprocesamiento de datasets
import pandas as pd
import sklearn as skl

#2. Contenido del dataset
En primer lugar, el contenido del dataset se carga en "dataframe", el cual Panda utilizará como unidad de información.\
El conjunto de datos consta de dos archivos:

mnist_train.csv\
mnist_test.csv\
El archivo mnist_train.csv contiene 60.000 ejemplos y etiquetas de entrenamiento. mnist_test.csv contiene 10.000 ejemplos de prueba y etiquetas. Cada fila consta de 785 valores: el primer valor es la etiqueta (un número del 0 al 9) y los 784 valores restantes son los valores de los píxeles (un número del 0 al 255).

In [80]:
dataframe = pd.read_csv('/content/drive/MyDrive/SIS 420/PrimerParcial2/mnist_train.csv')
print(dataframe)
print(dataframe.describe())

       label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0          5    0    0    0    0    0    0    0    0    0  ...      0      0   
1          0    0    0    0    0    0    0    0    0    0  ...      0      0   
2          4    0    0    0    0    0    0    0    0    0  ...      0      0   
3          1    0    0    0    0    0    0    0    0    0  ...      0      0   
4          9    0    0    0    0    0    0    0    0    0  ...      0      0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
59995      8    0    0    0    0    0    0    0    0    0  ...      0      0   
59996      3    0    0    0    0    0    0    0    0    0  ...      0      0   
59997      5    0    0    0    0    0    0    0    0    0  ...      0      0   
59998      6    0    0    0    0    0    0    0    0    0  ...      0      0   
59999      8    0    0    0    0    0    0    0    0    0  ...      0      0   

       28x21  28x22  28x23  28x24  28x2

#3.Preprocesamiento del dataset

###3.1. Preprocesamiento del dataset: Separar atributos(X) de la etiqueta (y) que se va a usar para el entrenamiento
Los atributos(X) se van a cargar en la variable "dataset" y la etiqueta "y" en la variable "label".

In [81]:
dataset = dataframe.drop("label", axis=1)
label = dataframe["label"].copy()

###3.2. Preprocesamiento del dataset: Atributos con texto
En nuestro dataset no tenemos atributos NO NUMÉRICOS.

###Limpieza de valores NaN en los atributos para evitar que los valores de theta salgan cero

In [82]:
print(dataframe.describe())

              label      1x1      1x2      1x3      1x4      1x5      1x6  \
count  60000.000000  60000.0  60000.0  60000.0  60000.0  60000.0  60000.0   
mean       4.453933      0.0      0.0      0.0      0.0      0.0      0.0   
std        2.889270      0.0      0.0      0.0      0.0      0.0      0.0   
min        0.000000      0.0      0.0      0.0      0.0      0.0      0.0   
25%        2.000000      0.0      0.0      0.0      0.0      0.0      0.0   
50%        4.000000      0.0      0.0      0.0      0.0      0.0      0.0   
75%        7.000000      0.0      0.0      0.0      0.0      0.0      0.0   
max        9.000000      0.0      0.0      0.0      0.0      0.0      0.0   

           1x7      1x8      1x9  ...         28x19         28x20  \
count  60000.0  60000.0  60000.0  ...  60000.000000  60000.000000   
mean       0.0      0.0      0.0  ...      0.200433      0.088867   
std        0.0      0.0      0.0  ...      6.042472      3.956189   
min        0.0      0.0      0

In [83]:
# Suponiendo que 'dataframe' es tu DataFrame
# Itera sobre cada columna y verifica si hay valores NaN
for col in dataframe.columns:
    # Verifica si hay valores NaN en la columna actual
    if dataframe[col].isnull().sum() > 0:
        # Imprime el nombre de la columna y la cantidad de valores NaN
        print(f"Categoría '{col}' tiene {dataframe[col].isnull().sum()} valores NaN.")

###Observación: No hay valores nan

## 4. Aplicación de la regresión logística onevsall
###Al inicio cargamos el dataframe en data.


In [84]:
# La entrada es de 785 elemnetos contando el X0
input_layer_size  = 785

# 10 etiquetas, de 0 a 9
num_labels = 10

#  datos de entrenamiento almacenados en los arreglos X, y
data = dataframe.copy()

# print(data['y'])
# print(data['y'].ravel())
# print(data['X'])
#X = data[:, :25]                                                #array de NumPy
#y = data[:, 25]

#Dataframe de Pandas
X = data.iloc[:, 1:785]  # Seleccionar todas las filas y las columnas de 1 a 784.
y = data.iloc[:, 0]   # Seleccionar todas las filas y solo la columna 0.

# X, y = data['X'], data['y'].ravel()
# print(X)
# print(y)
# establecer el dígito cero en 0, en lugar del 10 asignado a este conjunto de datos
# Esto se hace debido a que el conjunto de datos se utilizó en MATLAB donde no hay índice 0
#y[y == 10] = 0
#print(y)

m = y.size

In [85]:
#print(X[0,:])
#print(y)

#Dataframe de Pandas
print(X.iloc[1, :])               # Imprimir la primera fila de X,todas las columnas
print(y)                          # Imprimir la columna 'y' completa

1x1      0
1x2      0
1x3      0
1x4      0
1x5      0
        ..
28x24    0
28x25    0
28x26    0
28x27    0
28x28    0
Name: 1, Length: 784, dtype: int64
0        5
1        0
2        4
3        1
4        9
        ..
59995    8
59996    3
59997    5
59998    6
59999    8
Name: label, Length: 60000, dtype: int64


###Normalización

In [86]:
def featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)

    # Manejar el caso de desviación estándar igual a cero
    sigma[sigma == 0] = 1  # Si la desviación estándar es cero, establecerla a 1

    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [87]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [88]:
print(X_norm)

       1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  1x10  ...     28x19  \
0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
2      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
3      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
4      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...       ...   
59995  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
59996  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
59997  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
59998  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   
59999  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ... -0.033171   

          28x20     28x21     28x22     28x23    28x24  28x25  28x26  28x27  \
0     -0.022463 -0.0

In [89]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
#X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)                          #LO COMENTAMOS PORQUE ABAJO ESTAMOS VOLVIENDO A AGREGAR UNOS -> EN LA FUNCIÓN ONEVSALL
X = X_norm                                                                      #Y HACEMOS X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

### Vectorización de la función de costo

In [90]:
def sigmoid(z):                                                                 #SIGMOIDE
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [91]:
def lrCostFunction(theta, X, y, lambda_):                                       #FUNCIÓN DE COSTO
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

### Clasificacion One-vs-all   (con vectorización regularizada)


In [92]:
def oneVsAll(X, y, num_labels, lambda_):                                        #FUNCIÓN ONEVSALL
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [93]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(10, 785)


In [94]:
print(all_theta)

[[-8.211344    0.          0.         ...  0.          0.
   0.        ]
 [-9.78794502  0.          0.         ...  0.          0.
   0.        ]
 [-6.69767053  0.          0.         ...  0.          0.
   0.        ]
 ...
 [-8.25595221  0.          0.         ...  0.          0.
   0.        ]
 [-5.58849223  0.          0.         ...  0.          0.
   0.        ]
 [-7.5904924   0.          0.         ...  0.          0.
   0.        ]]



#### Prediccion One-vs-all

In [95]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

###Precisión del conjunto de entrenamiento

In [100]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)                                                        #LE PASAMOS LOS VALORES DE THETA Y LE PASAMOS EL VALOR DE X
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))

(60000, 784)
Precision del conjuto de entrenamiento: 92.73%


###PREDICCIONES

In [101]:
# Función de normalización
def featureNormalize(X):
    X_norm = X.copy()
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)

    # Manejar el caso de desviación estándar igual a cero
    sigma[sigma == 0] = 1  # Si la desviación estándar es cero, establecerla a 1

    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

# Cargar el dataset mnist_test.csv
mnist_test = pd.read_csv('/content/drive/MyDrive/SIS 420/PrimerParcial2/mnist_test.csv')

# Obtener las características (columnas 1 a 784) y las etiquetas (columna 0)
XPrueba = mnist_test.iloc[:, 1:].values
yPrueba = mnist_test.iloc[:, 0].values

# Normalizar las características
XPrueba, mu, sigma = featureNormalize(XPrueba)

# Agregar la columna de unos para el término de sesgo (bias)
XPrueba = np.concatenate([np.ones((XPrueba.shape[0], 1)), XPrueba], axis=1)

# Realizar predicciones
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)
print(p)  # Predicciones

# Mostrar los valores verdaderos
print(yPrueba)  # Etiquetas verdaderas

[7 2 1 ... 4 5 6]
[7 2 1 ... 4 5 6]
